# Introduction
This notebook implements an ETL Pipeline and Apache Cassandra (NoSQL) Database system to support data analysis for the Sparkify music streaming app, which stores all song and user activity data in CSV files. The ETL Pipeline extracts this data from the CSV files, creates the necessary tables designed to support several user-specified queries, inserts the required data into these tables, and creates/executes/verifies the desired queries.

# Part I. ETL Pipeline for Pre-Processing the Files

## Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

## Create list of filepaths to process original event csv data files

In [2]:
# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

## Process files in the data file csv that will populate Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

# Part II. Apache Cassandra - create the required tables & queries. 

#### Data contained in event_datafile_new.csv has the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

Screenshot of the denormalized data format of <font color=red>**event_datafile_new.csv**</font> :<br>

<img src="images/image_event_datafile_new.jpg">

## Create a Cluster

In [4]:
from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e)

## Create Keyspace

In [5]:
try:
    session.execute('''
                    CREATE KEYSPACE IF NOT EXISTS udacity
                    WITH REPLICATION = 
                    {'class' : 'SimpleStrategy', 'replication_factor' : 1}
                    ''')
except Exception as e:
    print(e)

## Set Keyspace

In [6]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

## CREATE QUERIES

### Query 1: 
Return artist name, song title & song length in the music app history associated with sessionId = 338 and itemInSession  = 4

In [7]:
query1c = "CREATE TABLE IF NOT EXISTS session_and_item" 
query1c = query1c + '''(
                   session_id int,
                   item_in_session int,
                   artist text,
                   song_title text,
                   song_length float,
                   PRIMARY KEY((session_id, item_in_session))
                   )'''

query1d = "DROP TABLE IF EXISTS session_and_item"

#### Drop any existing session_and_item table and create new table

In [8]:
# Drop session_and_item table (if exists)
try:
    session.execute(query1d)
except Exception as e:
    print(e)

# Create session_and_item table
try:
    session.execute(query1c)
except Exception as e:
    print(e)

#### Insert data into session_and_item table

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    query = "INSERT INTO session_and_item (session_id, item_in_session, artist, song_title, song_length)"
    query = query + "VALUES (%s, %s, %s, %s, %s)"
    for line in csvreader:
        try:
            session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))
        except Exception as e:
            print(e)

#### Perform a SELECT query to confirm proper insertion into the session_and_item table

In [10]:
query1 = "SELECT artist, song_title, song_length FROM session_and_item WHERE (session_id = 338) AND (item_in_session = 4)"

try:
    rows = session.execute(query1)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist, row.song_title, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


#### Verify the correctness of the above query

In [11]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        if (line[8] == str(338)) and (line[3] == str(4)):
            print(line)

['Faithless', 'Ava', 'F', '4', 'Robinson', '495.3073', 'free', 'New Haven-Milford, CT', '338', 'Music Matters (Mark Knight Dub)', '50']


### Query 2: 
Return artist name, song title (sorted by itemInSession) and user first & last name in the music app history associated with userId = 10 and sessionId = 182

In [12]:
query2c = "CREATE TABLE IF NOT EXISTS user_and_session"
query2c = query2c + '''(user_id int,
                        session_id int,
                        item_in_session int,
                        artist text,
                        song_title text,
                        first_name text,
                        last_name text,
                        PRIMARY KEY ((user_id, session_id), item_in_session)
                        ) WITH CLUSTERING ORDER BY (item_in_session ASC)
                    '''

query2d = "DROP TABLE IF EXISTS user_and_session"                    

#### Drop any existing user_and_session table and create new table

In [13]:
# Drop user_and_session table (if exists)
try:
    session.execute(query2d)
except Exception as e:
    print(e)

# Create user_and_session table
try:
    session.execute(query2c)
except Exception as e:
    print(e)

#### Insert data into user_and_session table

In [14]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    query = "INSERT INTO user_and_session (user_id, session_id, item_in_session, artist, song_title, first_name, last_name)"
    query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
    for line in csvreader:  
        try:
            session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))
        except Exception as e:
            print(e)

#### Perform a SELECT query to confirm proper insertion into the user_and_session table

In [15]:
query2 = "SELECT artist, song_title, first_name, last_name from user_and_session WHERE (user_id = 10) AND (session_id = 182)"

try:
    rows = session.execute(query2)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist, row.song_title, row.first_name, row.last_name)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


#### Verify the correctness of the above query

In [16]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        if (line[10] == str(10)) and (line[8] == str(182)):
            print(line)

['Down To The Bone', 'Sylvie', 'F', '0', 'Cruz', '333.76608', 'free', 'Washington-Arlington-Alexandria, DC-VA-MD-WV', '182', "Keep On Keepin' On", '10']
['Three Drives', 'Sylvie', 'F', '1', 'Cruz', '411.6371', 'free', 'Washington-Arlington-Alexandria, DC-VA-MD-WV', '182', 'Greece 2000', '10']
['Sebastien Tellier', 'Sylvie', 'F', '2', 'Cruz', '377.73016', 'free', 'Washington-Arlington-Alexandria, DC-VA-MD-WV', '182', 'Kilometer', '10']
['Lonnie Gordon', 'Sylvie', 'F', '3', 'Cruz', '181.21098', 'free', 'Washington-Arlington-Alexandria, DC-VA-MD-WV', '182', 'Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', '10']


### Query 3: 
Return user first & last name in the music app history for anyone who listened to song = 'All Hands Against His Own'

In [17]:
query3c = "CREATE TABLE IF NOT EXISTS specific_song"
query3c = query3c + '''
                      (song_title text,
                       user_id int,
                       first_name text,
                       last_name text,
                       PRIMARY KEY (song_title, user_id))
                    '''
 
query3d = "DROP TABLE IF EXISTS specific_song"

#### Drop any existing specific_song table and create new table

In [18]:
# Drop specific_song table (if exists)
try:
    session.execute(query3d)
except Exception as e:
    print(e)

# Create specific_song table
try:
    session.execute(query3c)
except Exception as e:
    print(e)

#### Insert data into specific_song table

In [19]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    query = "INSERT INTO specific_song (song_title, user_id, first_name, last_name)"
    query = query + "VALUES (%s, %s, %s, %s)"
    for line in csvreader:
        try:
            session.execute(query, (line[9], int(line[10]), line[1], line[4]))
        except Exception as e:
            print(e)

#### Perform a SELECT query to confirm proper insertion into the specific_song table

In [20]:
query3 = "SELECT first_name, last_name FROM specific_song WHERE song_title = 'All Hands Against His Own'"

try:
    rows = session.execute(query3)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.first_name, row.last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


#### Verify the correctness of the above query

In [21]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        if 'All Hands Against His Own' in line:
            print(line)

['The Black Keys', 'Tegan', 'F', '25', 'Levine', '196.91057', 'paid', 'Portland-South Portland, ME', '611', 'All Hands Against His Own', '80']
['The Black Keys', 'Sara', 'F', '31', 'Johnson', '196.91057', 'paid', 'Winston-Salem, NC', '152', 'All Hands Against His Own', '95']
['The Black Keys', 'Jacqueline', 'F', '50', 'Lynch', '196.91057', 'paid', 'Atlanta-Sandy Springs-Roswell, GA', '559', 'All Hands Against His Own', '29']


### Drop the tables before closing out the sessions

In [22]:
try:
    session.execute(query1d)
except Exception as e:
    print(e)

try:
    session.execute(query2d)
except Exception as e:
    print(e)
    
try:
    session.execute(query3d)
except Exception as e:
    print(e)

### Close the session and cluster connection

In [23]:
session.shutdown()
cluster.shutdown()